# COGS 108 - EDA Checkpoint

# Names
- Alan Miyazaki
- Alex Guan
- Nathan Ahmann
- Renaldy Herlim

<a id='research_question'></a>
# Research Question

Does crime happen more or less frequently around police stations and can that knowledge be used to more effectively distribute police station locations?

# Setup

In [ ]:
import pandas as pd
import datetime

# Data Cleaning

Describe your data cleaning steps here.

In [ ]:
# Reading datasets from two different time periods

# https://data.lacity.org/Public-Safety/Crime-Data-from-2010-to-2019/63jg-8b9z
past_df = pd.read_csv("Crime_Data_from_2010_to_2019.csv", dtype = {"TIME OCC": str})
# https://data.lacity.org/Public-Safety/Crime-Data-from-2020-to-Present/2nrs-mtv8
present_df = pd.read_csv("Crime_Data_from_2020_to_Present.csv", dtype = {"TIME OCC": str})

LAPD_df = pd.read_csv("LAPD_Police_Stations.csv")

# past dataset has column name typo
past_df = past_df.rename(columns={"AREA ": "AREA"})

# Both datasets use the same columns 
df = pd.concat([past_df, present_df])

In [ ]:
# Dropping code columns since these are internal use and we don't get much 
# value from them given we have their description in a seperate column
df = df.drop(columns=[
    "Crm Cd", "Crm Cd 1", "Crm Cd 2", "Crm Cd 3", 
    "Crm Cd 4", "Premis Cd", "Weapon Used Cd", 
    "Mocodes", "Part 1-2", "Status", "Date Rptd"
    ])

# Changed code to be readable description
df["Vict Descent"] = df["Vict Descent"].replace({
                            "A": "Other Asian", 
                            "B": "Black", 
                            "C": "Chinese", 
                            "D": "Cambodian", 
                            "F": "Filipino", 
                            "G": "Guamanian", 
                            "H": "Hispanic/Latin/Mexican", 
                            "I": "American Indian/Alaskan Native", 
                            "J": "Japanese", 
                            "K": "Korean", 
                            "L": "Laotian", 
                            "O": "Other", 
                            "P": "Pacific Islander", 
                            "S": "Samoan", 
                            "U": "Hawaiian", 
                            "V": "Vietnamese", 
                            "W": "White", 
                            "X": "Unknown", 
                            "Z": "Asian Indian",
                        })

# Renamed X,Y to Longitude, Latitude
LAPD_df = LAPD_df.rename(columns={
    "X": "Longitude", 
    "Y": "Latitude"
    })

# Data Analysis & Results (EDA)

Carry out EDA on your dataset(s); Describe in this section

In [ ]:
df.head()

One thing that we would be interested in knowing is if there is an equal distribution of crimes between police stations. From this we can see that there is a difference between how many cases each station has handled. This could either mean that some police stations are responsible for a larger area or there are more crimes happening in the areas those stations are covering. 

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
df["AREA NAME"].value_counts().plot(kind="barh")

Analyzing types of crime

In [ ]:
df["Crm Cd Desc"].unique()

In [ ]:
print(len(df["Crm Cd Desc"].unique()))

In [ ]:
#Analyze some common types of crimes

df['CRIME_TYPE_GROUP'] = df['Crm Cd Desc'].apply(lambda x: 'Petty' if 'PETTY' in x 
                                        else ('Child' if 'CHILD' in x 
                                        else ('Sexual' if 'SEX' in x 
                                        else ('Battery' if 'BATTERY' in x 
                                        else ('Assault' if 'ASSAULT' in x 
                                        else 'Other')))))

In [ ]:
df['CRIME_TYPE_GROUP'].value_counts()

Analyze the demographic of victims

In [ ]:
df_age = df[df['Vict Age'] > 0 ]['Vict Age']
df_age.describe()

In [ ]:



plt.bar(df_age.value_counts().index, df_age.value_counts().values)

# Add labels and a title
plt.xlabel('Age')
plt.ylabel('Count')
plt.title('Counts of Crime Victims by Age')

In [ ]:
df_vict_s = df[(df['Vict Sex'] == 'M')| (df['Vict Sex'] == 'F')]['Vict Sex']

plt.bar(df_vict_s.value_counts().index, df_vict_s.value_counts().values)

# Add labels and a title
plt.xlabel('Victim Sex')
plt.ylabel('Count')
plt.title('Counts of Crime Victims by Sex')

There are more Male victims than Female victims

In [ ]:
df_vict_d = df['Vict Descent']

In [ ]:
plt.bar(df_vict_d.value_counts().index, df_vict_d.value_counts().values)

# Add labels and a title
plt.xlabel('Victim Race')
plt.xticks(rotation = 90)
plt.ylabel('Count')
plt.title('Counts of Crime Victims by Race')

## Time of Crime Analysis

In [ ]:
df["TIME OCC"].value_counts()

This is to see when do most crimes happen. From this we can see that the most common hour for a crime to occur over the past 13 years was at noon. The least common time for a crime to occur was at 5 am. However this is a massive spike at 12 which is odd. A reason for this could be that the dataset only counts crimes reported which means that since it is noon, more people are out and can see/report on a crime. In contrast, people are sleeping at night so there are less people reporting crimes. Another reason for this could be that when a crime happens, they are more focused on the criminal instead of what time it is so when they make a report to an officer, they may choose noon due to it being daylight.

In [ ]:
df["TIME OCC"].str[:2].value_counts().sort_index().plot()

In [ ]:
df["TIME OCC"].str[:2].value_counts()

Group times into "morning" , "afternoon", "evening", and "night"

In [ ]:
df["TIME OCC"].str[:2].astype(int)

In [ ]:
# Define a dictionary to map the times to time of day categories
time_dict = {'Morning': range(5, 12),
             'Afternoon': range(12, 18),
             'Evening': range(18, 24),
             'Night': range(0,5)}

# Use the pandas map function to create a new column with the time of day categories
df['TIME_OCC_GROUP'] = df["TIME OCC"].str[:2].astype(int).map(lambda x: next((k for k, v in time_dict.items() if int(x) in v), None))

In [ ]:
df['TIME_OCC_GROUP'].value_counts()

In [ ]:
df['TIME_OCC_GROUP'].value_counts().values

In [ ]:
# Create a barplot using matplotlib
plt.bar(df['TIME_OCC_GROUP'].value_counts().index, df['TIME_OCC_GROUP'].value_counts().values)

# Add labels and a title
plt.xlabel('Time of Day')
plt.ylabel('Count')
plt.title('Counts of Crime by Time of Day')

# Show the plot
plt.show()

#### Explore time of crime and demographic variables

In [ ]:
df_sex = df[(df['Vict Sex'] == 'M')| (df['Vict Sex'] == 'F')]
df_sex = pd.DataFrame(df_sex.groupby(['Vict Sex', 'TIME_OCC_GROUP']).count()['TIME OCC'])

In [ ]:
print(df_sex)

In [ ]:
df["DATE OCC"].value_counts().sort_index()

In [ ]:
pd.to_datetime(df["DATE OCC"]).value_counts().sort_index().plot()

Seems like there are more crimes happening during new years?

In [ ]:
df.head()

In [ ]:
pd.to_datetime(df["DATE OCC"]).value_counts()

Number of crimes reported plotted by month and year

In [ ]:
pd.to_datetime(df["DATE OCC"]).dt.to_period('M').value_counts().sort_index().plot()

In [ ]:
pd.to_datetime(df["DATE OCC"]).dt.to_period('M').value_counts()

In which month of the year is crime typically higher?

In [ ]:
months = pd.to_datetime(df["DATE OCC"]).dt.month

# Create a barplot using matplotlib
plt.bar(months.value_counts().index, months.value_counts().values)

# Add labels and a title
plt.xlabel('Month')
plt.ylabel('Count')
plt.title('Counts of Crime by Month')

# Show the plot
plt.show()

It looks like crime is pretty much equally spread out throughout the year, except for January.

This is number of crims reported over years. 2023 is extremely low because the year is going on.

In [ ]:
pd.to_datetime(df["DATE OCC"]).dt.to_period('Y').value_counts().sort_index().plot()

In [ ]:
pd.to_datetime(df["DATE OCC"]).dt.to_period('Y').value_counts()

Removing 2023 to get a better graph. We can see some oddities. 2013, 2014, and 2020 are low in comparison to other years. 2020 is likely due to the pandemic since everyone was encouraged to stay at home which leads to less crimes reported (and maybe less crimes happening?). I have no idea of any reason to explain why 2013 and 2014 are so low. In addition its a bit weird that crime starts picking to make a curve at 2017. Also a massive spite at 2022 as well but not too sure of the reason there. 

In [ ]:
pd.to_datetime(df["DATE OCC"]).dt.to_period('Y').value_counts().sort_index()[:-1].plot()

In [ ]:
LAPD_df.head()

## Looking at New Years data from 2010 to 2023

In [ ]:
# Make a copy of the dataframe and create a new one with a datetime column for easy filtering
datetime_col = pd.to_datetime(df['DATE OCC'])
new_df = df.copy()
new_df['Date'] = datetime_col
new_df.head()

In [ ]:
# Create new dataframe that only contains crimes on New Years Day from 2010 - 2023
new_years = new_df[(new_df['Date'].dt.month == 1) & (new_df['Date'].dt.day == 1)]
new_years.head()

In [ ]:
# Over 20,000 crimes committed on the 13 New Years day's alone!
new_years.shape

#### 13 days are accounting for over 20,000 crimes in total, with an average of 1558 per day

In [ ]:
20262/13

In [ ]:
365 * 12 + (80)

In [ ]:
(df.shape[0] / 4460) * 13

In [ ]:
new_years['Vict Descent'].value_counts()

In [ ]:
top_20 = new_years['Crm Cd Desc'].value_counts()[:20]
top_20

In [ ]:
plt.bar(top_20.index, top_20)
plt.xticks(rotation = 90)
plt.show()

In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt

# https://geohub.lacity.org/datasets/031d488e158144d0b3aecaa9c888b7b3_0/explore?location=33.985210%2C-118.389876%2C11.32

map_df = gpd.read_file('LAPD_Divisions.shp')
map_df.plot(color='gray')

df_plot = df.loc[(df['LAT'] >= 33.7) & (df['LAT'] <= 34.37) & (df['LON'] >= -118.7) & (df['LON'] <= -118.15)]

fig, ax = plt.subplots(1, figsize = (20, 12))
map_df.boundary.plot(color='black', ax=ax)
df_plot.plot(ax=ax, x='LON', y='LAT', kind='scatter', color='red',  s=0.1, alpha=0.5)
LAPD_df.plot(ax=ax, x='Longitude', y='Latitude', kind='scatter', s=5)

LAPD_df.merge(df.groupby('AREA')[['DR_NO']].count(), left_on='PREC', right_on='DR_NO')